In [3]:
import os
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
import re

import evaluate
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModelForCausalLM
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

# konlpy
import konlpy
from konlpy.tag import Okt

# Seed Set
SEED = 456
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

# 디바이스 설정 (GPU가 사용 가능하면 GPU를 사용하고, 그렇지 않으면 CPU 사용)
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [4]:
data_path = os.path.join('..', '..', 'data','v0.1')
data = pd.read_csv(os.path.join(data_path,"relabeled_using_SBERT.csv"))
data = data.drop(columns=["Unnamed: 0"])
print(data_path)

../../data/v0.1


### Calc Ratio

In [42]:
text_datas = data["text"]
def calculate_ascii_noise(text):
    total_chars = len(text)
    ascii_chars = 0 
    for ch in text:
        if ch == " ":
            continue
        if ord(ch) < 128:
            ascii_chars+=1
            
    non_ascii_chars = total_chars - ascii_chars  # 비-ASCII 문자 개수
    
    ascii_ratio = (ascii_chars / total_chars) * 100  # ASCII 문자 비율 (%)
    
    return ascii_chars, non_ascii_chars, ascii_ratio
asc_counts = []; non_asc_counts = []; percentages = []
# ASCII 문자 개수, 비-ASCII 문자 개수, ASCII 비율 계산
for text_with_noise in text_datas:
    ascii_count, non_ascii_count, ascii_percentage = calculate_ascii_noise(text_with_noise)
    ascii_percentage = round(ascii_percentage,2)
    asc_counts.append(ascii_count)
    non_asc_counts.append(non_ascii_count)
    percentages.append(ascii_percentage)
    
print(len(asc_counts), len(non_asc_counts), len(percentages))
data["ratio"] = percentages

1504 1504 1504


In [44]:
Noise_data = data[data["ratio"]>25]
Noise_data

,ID,text,target,ratio
14,ynat-v1_train_00033,학~ 석면 S거 규정k지키5,3,26.67
29,ynat-v1_train_00066,UEFA 챔스리그 4강 마드리드 더비 성사…레알 vs 아...,1,29.41
35,ynat-v1_train_00076,北조국통!민주X#전선 결성 <0돌 중;보고회 열x,2,25.93
60,ynat-v1_train_00126,날씨-미z먼지 보통 수y…오후부터 xJ지고:곳곳 F12일,0,29.03
65,ynat-v1_train_00138,삼성 빅스. 개발자데KS1B월 서울B 첫 +최,4,28.00
...,...,...,...,...
1473,ynat-v1_train_02739,:한금투 자동차 업/ 1분기부k5실적 점진?|I선,5,29.63
1474,ynat-v1_train_02741,~ 전 앞둔eW재 감98T이 부K되지만 4로 승부수,1,27.59
1476,ynat-v1_train_02745,軍v<들 전용3량 타고 골l장y…`덕적 해R 심각,3,25.93
1491,ynat-v1_train_02780,"北0대! 김정*2향한 ""성경쟁 무대된l릴레이r토론",6,25.93


### BART 방식

In [27]:
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration
#  Load Model and Tokenize
tokenizer = PreTrainedTokenizerFast.from_pretrained("jian1114/jian_KoBART_title")
model = BartForConditionalGeneration.from_pretrained("jian1114/jian_KoBART_title")

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [28]:
print(tokenizer.all_special_ids)
print(tokenizer.all_special_tokens)

[0, 1, 5, 3, 6]
['<s>', '</s>', '<unk>', '<pad>', '<mask>']


In [29]:
def replace_ascii_with_mask(text):
    # 결과를 저장할 리스트
    result = []
    
    # 입력 문자열을 한 문자씩 순회
    for char in text:
        # 공백은 그대로 추가
        if char == ' ':
            result.append(char)
        # ASCII 코드 범위(0 ~ 127) 확인, 공백 제외
        elif ord(char) <= 127:
            # ASCII 문자면 <mask>로 대체
            result.append('<mask>')
        else:
            # 그렇지 않으면 원래 문자를 그대로 추가
            result.append(char)
    
    # 리스트를 문자열로 변환하여 반환
    return ''.join(result)

In [30]:
input_text = "서울문고 N풍그U에 인수… f라인서점 교m영풍y양강d도로"
# input_ids = tokenizer(input_text, return_tensors='pt')['input_ids']
input_ids = tokenizer(replace_ascii_with_mask(input_text), return_tensors='pt')['input_ids']

In [31]:
print(input_ids)
output_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
output_text

tensor([[14245, 10671,  9102,  1700,     6, 15046,  9229,     6, 14364, 15862,
         14476,  1700,     6, 20835, 11264, 12136, 14139,     6, 14126, 13517,
             6, 14256,  9000,     6, 16522]])


'서울문고  풍그 에 인수...  라인서점 교 영풍 양강 도로'

In [33]:
output_ids = model.generate(
    input_ids, 
    max_length=50, 
    early_stopping=True,
    num_beams=3,
    no_repeat_ngram_size=2,  # 3-그램 이상의 반복 방지
    temperature=0.7,         # 다양성 조절
    top_p=0.92,
    top_k=50
)

# 복원된 텍스트 디코딩
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(output_text)

서울문고 랜드마크, 풍그래픽스위크 )는 서울문화재단, 서울양강로(강남구 양천구 소재) 뷰어센티움, 영풍문고, 양강도서관,


### Prompting

In [65]:
text_datas = Noise_data["text"]

In [48]:
# rtzr/ko-gemma-2-9b-it

model_id = "rtzr/ko-gemma-2-9b-it"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [59]:
few_shot_list = [
    {
        "input":"서울문고 N풍그U에 인수… f라인서점 교m영풍y양강d도로",
        "output":"서울문고, 영풍그룹에 인수…오프라인서점 교보-영풍 양강구도로"
    },
    {
        "input":"프로야구~롯TKIAs광주 경기 y천취소",
        "output":"프로야구 롯데-KIA 광주 경기 우천취소"
    },
    {
        "input":"러시아 시리아서 S500 발사했나…러 국방부는 부인",
        "output":"러시아 시리아서 S500 발사했나…러 국방부는 부인"
    },
    {
        "input":"xW리 a)엔 예비후0V사전여론조사 결과 유출 논c",
        "output":"새누리, 이번엔 예비후보 사전여론조사 결과 유출 논란"
    },
    {
        "input":"조국U압수수색f검2와 통7 파장…한h·바(미래 &핵K진i",
        "output":"조국 '압수수색 검사와 통화' 파장…한국·바른미래 '탄핵추진'"
    }
]

In [62]:
input_text = "삼성 빅스. 개발자데KS1B월 서울B 첫 +최"
# input_text = replace_ascii_with_mask(input_text)

system_message = """당신은 뉴스 기사 제목에 있는 노이즈를 제거하는 어시스턴트로, 한 문장 내외의 뉴스 기사 제목을 복원해야합니다. 다음 지침을 따르세요.

1. 제목에 오류가 없다면 수정하지 마세요.
2. 원본 제목의 의미, 뉘앙스, 형식을 최대한 유지하세요.
3. 오타, 특수문자 오용, 인코딩 오류 등을 수정하세요.
4. 제목이 이해하기 어렵다면 명확하게 다듬으세요.
"""
few_shot_messages = []
for few_shot_data in few_shot_list:
    few_shot_messages.append(
        {"role":"user","content":few_shot_data["input"]}
    )
    few_shot_messages.append(
        {"role":"assistant","content":few_shot_data["output"]}
    )
messages = [
    [{"role":"system", "content":system_message}] +
    few_shot_messages +
    [{"role":"user", "content":input_text}]
]
print(messages)

[[{'role': 'system', 'content': '당신은 뉴스 기사 제목에 있는 노이즈를 제거하는 어시스턴트로, 한 문장 내외의 뉴스 기사 제목을 복원해야합니다. 다음 지침을 따르세요.\n\n1. 제목에 오류가 없다면 수정하지 마세요.\n2. 원본 제목의 의미, 뉘앙스, 형식을 최대한 유지하세요.\n3. 오타, 특수문자 오용, 인코딩 오류 등을 수정하세요.\n4. 제목이 이해하기 어렵다면 명확하게 다듬으세요.\n'}, {'role': 'user', 'content': '서울문고 N풍그U에 인수… f라인서점 교m영풍y양강d도로'}, {'role': 'assistant', 'content': '서울문고, 영풍그룹에 인수…오프라인서점 교보-영풍 양강구도로'}, {'role': 'user', 'content': '프로야구~롯TKIAs광주 경기 y천취소'}, {'role': 'assistant', 'content': '프로야구 롯데-KIA 광주 경기 우천취소'}, {'role': 'user', 'content': '러시아 시리아서 S500 발사했나…러 국방부는 부인'}, {'role': 'assistant', 'content': '러시아 시리아서 S500 발사했나…러 국방부는 부인'}, {'role': 'user', 'content': 'xW리 a)엔 예비후0V사전여론조사 결과 유출 논c'}, {'role': 'assistant', 'content': '새누리, 이번엔 예비후보 사전여론조사 결과 유출 논란'}, {'role': 'user', 'content': '조국U압수수색f검2와 통7 파장…한h·바(미래 &핵K진i'}, {'role': 'assistant', 'content': "조국 '압수수색 검사와 통화' 파장…한국·바른미래 '탄핵추진'"}, {'role': 'user', 'content': '삼성 빅스. 개발자데KS1B월 서울B 첫 +최'}]]


In [69]:
def generate_sentence(messages, inp_text):
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<end_of_turn>")
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=1024,
        eos_token_id=terminators,
        do_sample=False,
        temperature=0.6,
        top_p=0.9,
    )
    decoded_data = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)
    generate_data.append(decoded_data)
    print("RAW :",inp_text)
    print("GEN :",decoded_data)
generate_sentence(messages=messages, inp_text=input_text)

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


NameError: name 'generate_data' is not defined

In [70]:
generate_data = []
few_shot_messages = []
for few_shot_data in few_shot_list:
    few_shot_messages.append(
        {"role":"user","content":few_shot_data["input"]}
    )
    few_shot_messages.append(
        {"role":"assistant","content":few_shot_data["output"]}
    )
for inp_text in text_datas:
    messages = [
        [{"role":"system", "content":system_message}] +
        few_shot_messages +
        [{"role":"user", "content":inp_text}]
    ]
    generate_sentence(messages, inp_text)

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


RAW : 학~ 석면 S거 규정k지키5
GEN : 학생 석면 노출 규정 지키기 




RAW : UEFA 챔스리그 4강 마드리드 더비 성사…레알 vs 아...
GEN : UEFA 챔스리그 4강, 마드리드 더비 성사…레알 vs 아틀레티코  

RAW : 北조국통!민주X#전선 결성 <0돌 중;보고회 열x
GEN : 북조국, 통일민주전선 결성 발표…보고회 열려  

RAW : 날씨-미z먼지 보통 수y…오후부터 xJ지고:곳곳 F12일
GEN : 날씨-미세먼지 보통 수준…오후부터 지속적인 고온, 곳곳 폭염 예상  




RAW : 삼성 빅스. 개발자데KS1B월 서울B 첫 +최
GEN : 삼성 빅스, 개발자 데모 10월 서울 첫 개최 예정  

RAW : MLB.com 다저스 3년 연속 WS 유력하지만 우승은 글쎄…
GEN : MLB.com: 다저스 3년 연속 WS 유력, 우승은 불투명  

RAW : 현대차 현대차증권 MMT 300억원 매수
GEN : 현대차, 현대차증권 MMT 300억원 매수 



RAW : 후랭코프 5승 두산 20승 선착…SK·LG 맹추격
GEN : 후랭코프 5승, 두산 20승 선착…SK·LG 맹추격  




RAW : 바르샤 이승우z)z·네h란드 팀으로k임대 가AG
GEN : 바르샤 이승우, 네덜란드 팀으로 임대 가  

RAW : 韓 물가 7c 초미세먼지-세계 최M…관Zugq 걸림돌
GEN : 한국의 물가 7% 초과, 미세먼지-세계 최고…관련 정책 걸림돌 

RAW : w대통령 호W,J거소@방문…호 주석 본F으k 부패 없어CN것
GEN : 대통령 호주, 거소 방문…호 주석 본인 부패 없었다고 주장 



RAW : 리J에겐 인격적 소S~력 중요…P과만중KA독불`군형은 NO
GEN : 리더에게는 인격적 소통력 중요…'과만 중심, 한국 독불군형은 NO' 



RAW : 아랍권 반D란"정상회의 5춰 (\s& 이란 맹9종합
GEN : 아랍권 반이란 "정상회의 5차례, 이란 맹렬 공격"  

RAW : q승하러 왔죠…전광인?우승 한 풀% U%P$

In [71]:
Noise_data["generated"] = generate_data

/tmp/ipykernel_1541094/2159022743.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Noise_data["generated"] = generate_data


In [72]:
Noise_data.to_csv("./Noise_filtering_gemma.csv",encoding="utf-8-sig")

In [6]:
# 원래 데이터와 병합
data

,ID,text,target
0,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,4
1,ynat-v1_train_00005,美성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다,3
2,ynat-v1_train_00007,아가메즈 33득점 우리카드 KB손해보험 완파…3위 굳...,1
3,ynat-v1_train_00008,朴대통령 얼마나 많이 놀라셨어요…경주 지진현장 방문종합,2
4,ynat-v1_train_00009,듀얼심 아이폰 하반기 출시설 솔솔…알뜰폰 기대감,4
...,...,...,...
1499,ynat-v1_train_02794,문 대통령 김기식 금감원장 사표 수리키로종합,2
1500,ynat-v1_train_02795,트럼프 폭스뉴스 앵커들 충성도 점수매겨…10점만점에 12점도,6
1501,ynat-v1_train_02796,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,4
1502,ynat-v1_train_02798,인터뷰 류현진 친구에게 안타 맞는 것 싫어해…승부는 냉정,1


In [7]:
Noise_data_removed = pd.read_csv(os.path.join(data_path, "Noise_filtering_gemma.csv"))

In [9]:
Noise_data_removed[["ID","generated"]]

,ID,generated
0,ynat-v1_train_00033,학생 석면 노출 규정 지키기 \n\n\n\n
1,ynat-v1_train_00066,"UEFA 챔스리그 4강, 마드리드 더비 성사…레알 vs 아틀레티코 \n"
2,ynat-v1_train_00076,"북조국, 통일민주전선 결성 발표…보고회 열려 \n"
3,ynat-v1_train_00126,"날씨-미세먼지 보통 수준…오후부터 지속적인 고온, 곳곳 폭염 예상 \n\n\n\n"
4,ynat-v1_train_00138,"삼성 빅스, 개발자 데모 10월 서울 첫 개최 예정 \n"
...,...,...
138,ynat-v1_train_02739,한금투 자동차 업계 1분기 부실적 점진적 회복 예상 \n\n\n
139,ynat-v1_train_02741,"전 앞둔 K리그, 감독이 부르지만 4-4로 승부수 \n"
140,ynat-v1_train_02745,군인들 전용 3량 타고 골프장…'덕적 해 심각' \n\n\n\n
141,ynat-v1_train_02780,"북한대! 김정은 ""성경쟁 무대 된 리레이 토론"" \n\n\n\n"


In [ ]:
Noise_data_removed["generated"].apply(lambda x: x.replace("\n",""))

0                            학생 석면 노출 규정 지키기 \n\n\n\n
1            UEFA 챔스리그 4강, 마드리드 더비 성사…레알 vs 아틀레티코  \n
2                        북조국, 통일민주전선 결성 발표…보고회 열려  \n
3      날씨-미세먼지 보통 수준…오후부터 지속적인 고온, 곳곳 폭염 예상  \n\n\n\n
4                    삼성 빅스, 개발자 데모 10월 서울 첫 개최 예정  \n
                            ...                      
138               한금투 자동차 업계 1분기 부실적 점진적 회복 예상 \n\n\n
139                    전 앞둔 K리그, 감독이 부르지만 4-4로 승부수 \n
140               군인들 전용 3량 타고 골프장…'덕적 해 심각' \n\n\n\n
141              북한대! 김정은 "성경쟁 무대 된 리레이 토론"  \n\n\n\n
142                이총리, 4월 H직 완전히 못해할 일 꽤 남아 \n\n\n\n
Name: generated, Length: 143, dtype: object